# Family counts per year

By **Franklin Oliveira**

-----
This notebook contains all code necessary to make the "type" charts from `carcinos` database. Here you'll find some basic data treatment and charts' code. 

Database: <font color='blue'>'Planilha geral Atualizada FINAL 5_GERAL_sendo trabalhada no Google drive.xlsx'</font>

In [1]:
import datetime
import numpy as np
import pandas as pd

from collections import defaultdict

# quick visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Waffle Charts
# from pywaffle import Waffle 
# docs: https://pywaffle.readthedocs.io/en/latest/examples/block_shape_distance_location_and_direction.html

# visualization
import altair as alt

# enabling notebook renderer
# alt.renderers.enable('notebook')
alt.renderers.enable('default')

# disabling rows limit
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Importing data...

In [2]:
NewTable = pd.read_csv('./data/treated_db.csv', sep=';', encoding='utf-8-sig', low_memory=False)

## Filtering

At least for now, we'll be considering only specimens of order decapoda (deeply revised by the Museum's crew)

In [3]:
decapoda = NewTable[NewTable['order'] == 'Decapoda'].copy()

In [9]:
# correction asked by Cristiana Serejo (there infraorders should be reported as Brachyura)
errors = ['Grapsoidea', 'Xanthoidea']
correction = 'Brachyura'

decapoda['infraorder'] = decapoda['infraorder'].apply(lambda x: correction if str(x) in errors else x)

<br>

<font size=5>**Color palette**</font>

Colors (per infraorder): 

- <font color='#e26d67'><b>Ascacidae</b></font>
- <font color='#007961'><b>Anomura</b></font>
- <font color='#7a2c39'><b>Achelata</b></font>
- <font color='#b67262'><b>Axiidea</b></font>
- <font color='#ee4454'><b>Brachyura</b></font>
- <font color='#3330b7'><b>Caridea</b></font>
- <font color='#58b5e1'><b>Gebiidea</b></font>
- <font color='#b8e450'><b>Stenopodídea</b></font>
- <font color='#a0a3fd'><b>Astacidae</b></font>
- <font color='#deae9e'><b>Polychelida</b></font>
- <font color='#d867be'><b>Grapsidae</b></font>
- <font color='#fece5f'><b>Xanthoidea</b></font>

In [10]:
# importing customized color palettes
from src.MNViz_colors import *

<br>


## Graphs

---
### Creating chart: counts per order per year

In [11]:
infraorders = decapoda.groupby(['start_year','infraorder', 'family']).count()['class'].reset_index().rename(columns={'class':'counts'})

infraorders.sort_values(['start_year','infraorder'], inplace=True)  # ordering

In [12]:
# dropping remaining NaN's
# infraorders = infraorders.dropna(subset=['infraorder'])

In [15]:
# database 
db = infraorders

# aux. variables
x_labels = db.sort_values('start_year')['start_year'].unique()
y_labels = db.groupby('infraorder').sum().reset_index().sort_values('counts')['infraorder'].unique()[::-1]
temp = db.sort_values('counts')['counts'].unique()
counts = list(range(min(temp), max(temp), 20))

# selector
select_family = alt.selection_multi(fields=['family'], bind='legend')

# chart
g1 = alt.Chart(db, width=800, height=500, 
               title='Total of decapoda collected per infraorder each year').mark_circle().encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year',
             scale= alt.Scale(domain=x_labels)),
    y= alt.Y('infraorder', type='nominal', title='Infraorder',
             scale= alt.Scale(domain= y_labels),
             sort= alt.EncodingSortField(field='counts', op='sum', order='descending')),
    size = alt.Size('counts', title='Counts', 
                    scale=alt.Scale(domain= counts, range=[20,120]),
                    legend= alt.Legend(orient= 'right', direction='horizontal')),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color = alt.Color('family:N', title='Family', 
                      scale= alt.Scale(domain=list(cores_familia_naive.keys()), 
                                       range=list(cores_familia_naive.values())),
                      legend= alt.Legend(columns=3, symbolLimit=102)),
    tooltip= alt.Tooltip(['start_year', 'infraorder', 'family', 'counts'])
).add_selection(select_family).transform_filter(select_family)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# saving graph
# g1.save('./graphs/infraorders_per_year.html')
# g1

### number of decapodas per family per year

In [9]:
teste = decapoda.groupby(['family','start_year']).count()['class'].reset_index().rename(
                                                                                    columns={'class':'counts'})

# teste['start_year'] = teste['start_year'].astype(int)  # there are non-integers remaining

<br>

**graph:** family per year

In [11]:
g1 = alt.Chart(teste,
               width=500, height=1000, title='Total of decapoda collected each year').mark_circle(
                                                                                size=60).encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('family', type='nominal', title='Family',
            sort= alt.EncodingSortField(field='counts', op='count', order='descending')),
    size= alt.Size('counts', title='Count', scale=alt.Scale(range=[6,400])),
    tooltip = alt.Tooltip(['family', 'start_year', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/families_per_year.html')
# g1

In [12]:
teste = NewTable.groupby(['family','infraorder','start_year']).count()['class'].reset_index().rename(
                                                                                    columns={'class':'counts'})

# teste['start_year'] = teste['start_year'].astype(int) # there are non-integers remaining

In [14]:
g1 = alt.Chart(teste,
               width=500, height=1200, title='Total of decapoda collected per family each year').mark_circle(
                                                                                size=60).encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('family', type='nominal', title='Family',
            sort= alt.EncodingSortField(field='counts', op='sum', order='descending')),
    size= alt.Size('counts', title='Counts', scale=alt.Scale(range=[6,400]),
                   legend= alt.Legend(orient= 'right', direction='horizontal')),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color = alt.Color('family:N', title='Family', 
                      scale= alt.Scale(domain=list(cores_familia_naive.keys()), 
                                       range=list(cores_familia_naive.values())),
                      legend= alt.Legend(columns=2, symbolLimit=102)),
    tooltip = alt.Tooltip(['family', 'start_year', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/families_per_year-colorful-FULL.html')
# g1

In [19]:
fam_legend = teste['family'].unique()
fam_legend = [f for f in fam_legend if f in cores_familia_naive.keys()]
g1 = alt.Chart(teste[teste['family'].isin(fam_legend)],
               width=500, height=1200, title='Total of decapoda collected per family each year').mark_circle(
                                                                                size=60).encode(
    x= alt.X('start_year', type='ordinal', title='Sampling Year'),
    y= alt.Y('family', type='nominal', title='Family',
            sort= alt.EncodingSortField(field='counts', op='sum', order='descending')),
    size= alt.Size('counts', title='Count', scale=alt.Scale(range=[6,400]),
                   legend= alt.Legend(columns=5)),
    order= alt.Order('counts', sort='descending'),  # smaller points in front
    color = alt.Color('family:N', title='Family', 
                      scale= alt.Scale(domain=list(cores_familia_naive.keys()), 
                                       range=list(cores_familia_naive.values())),
                      legend= alt.Legend(columns=2, symbolLimit=102)),
    tooltip = alt.Tooltip(['family', 'start_year', 'counts'])
)

g1 = g1.configure_title(fontSize=16).configure_axis(
    labelFontSize=12,
    titleFontSize=12
).configure_legend(
    labelFontSize=12,
    titleFontSize=12
)

# g1.save('./graphs/families_per_year-colorful.html')
# g1

<br>

**The end!**

-----